# Resistance Measurements of 2nd Batch of Sponge-Wedge Chips

## Summary Graphs

### Data
Electrical resistances from a total of n=3 chips were measured. 

Data are organized in dictionaries of repeated measurements of averages and standard deviations of 100 observations for each channel (using the Keithley Multimeter device). Each value is the average and the standard deviation of the 100 observations. Keys of the dictionary are protocols, and value is the data (which is a dictionary, too). The data dictionary is 

In [4]:
import os
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [2]:
os.getcwd()

'/home/mkrause/dev/biofuel-cell/notebooks'